In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
import time

sys.path.append(os.path.join(os.environ['REPO_DIR'], 'utilities'))
from utilities2015 import *
from metadata import *
from data_manager import *

from visualization_utilities import *

from multiprocess import Pool

Setting environment for Gordon


No vtk


In [2]:
stack = 'MD642'

In [4]:
classifier_setting = 12

# add_annotation = True
downscale = 8

In [7]:
sec = 193
viz = scoremap_overlay(stack=stack, sec=sec, structure='7N', downscale=downscale, 
                       setting=resolve_actual_setting(setting=classifier_setting, stack=stack, sec=sec))

/oasis/projects/nsf/csd395/yuncong/brain_virtualenv/lib/python2.7/site-packages/skimage/util/dtype.py:122: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))


In [8]:
display_image(viz, filename='tmp.png')

/oasis/projects/nsf/csd395/yuncong/Brain/learning/tmp.png

In [9]:
first_sec, last_sec = metadata_cache['section_limits'][stack]

In [ ]:
# for sec in range(200, 202):
# for sec in [230, 231]:
for sec in range(first_sec, last_sec+1):
    
    print sec
    
    if is_invalid(stack=stack, sec=sec):
        continue
    
    actual_setting = resolve_actual_setting(setting=classifier_setting, stack=stack, sec=sec)
    
    t = time.time()
    
    bg = imread(DataManager.get_image_filepath(stack=stack, section=sec, resol='lossless', version='compressed'))
    
    def f(structure):
        viz_fp = DataManager.get_scoremap_viz_filepath(stack=stack, section=sec, structure=structure, setting=actual_setting)
        try:
            viz = scoremap_overlay_on(bg=bg, stack=stack, sec=sec, structure=structure, 
                                      downscale=downscale, label_text=None, setting=actual_setting)
            create_if_not_exists(os.path.dirname(viz_fp))
            imsave(viz_fp, img_as_ubyte(viz))
        except Exception as e:
            sys.stderr.write('%s\n' % e)
            return
    
    pool = Pool(8)
    pool.map(f, all_known_structures)
    pool.close()
    pool.join()

    sys.stderr.write('Visualize scoremaps: %.2f seconds.\n' % (time.time() - t)) 
    # 7s for one structure, one section, single process
    # 20s for all structures, one section, 8 processes

In [ ]:
# if not add_annotation:
# export_scoremaps_multiprocess(bg='original', stack=stack, sections=range(200, 201),
#                               structures=all_known_structures, downscale_factor=8,
#                               label_text=True, setting=setting) # 93s/section
# else:
#     outputViz_rootdir = '/oasis/projects/nsf/csd395/yuncong/CSHL_scoremapPlusAnnotationViz'
#     export_filepath_fmt = outputViz_rootdir + '/%(name)s/%(stack)s/%(fn)s_alignedTo_%(anchor_fn)s_scoremapPlusAnnotationViz_%(name)s_%(annofn)s.jpg' % dict(anchor_fn=anchor_fn)
#     export_scoremapPlusAnnotationVizs('original', stack, range(first_sec, last_sec+1),
#                                       structures, 8, export_filepath_fmt=export_filepath_fmt,
#                                       users=['yuncong', 'localAdjusted', 'autoAnnotate', 'globalAligned'])
